In [2]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

In [3]:
bestlcs=pd.read_csv('makingtable/bestlcs.csv')
bestlclist=np.array([], dtype=object)
start_col=bestlcs.columns.get_loc('Sector 1')
for index, row in bestlcs.iterrows(): #iterate through each object
    for col in bestlcs.columns[start_col:]: #iterate through all the columns starting with the sectors
        value=row[col]
        if not pd.isna(value):
            bestlclist=np.append(bestlclist, [f'{row['pop_id']}_{value}'])

In [4]:
K2SFFlist=np.loadtxt('makingtable/K2SFFlist.txt', dtype=str)
lclist=pd.concat([pd.DataFrame(bestlclist), pd.DataFrame(K2SFFlist)], ignore_index=True)
lclist[['pop_id', 'lcname']]=lclist[0].str.split('_', expand=True)
lclist.drop(columns=0, inplace=True)

In [5]:
lclist['complex_status']=""

In [10]:
for index, currow in lclist.iterrows(): 
    curlc=get_lc_string(currow['lcname'])
    if currow['complex_status']!="":
        continue
    try:
        complexity,_= harmonic_checker(curlc,max_period=5,percentage=0.10)
        lclist.at[index,'complex_status']=complexity
    except Exception as e:
        print(f'Error getting complex status for {currow['lcname']}: {e}')

Error getting complex status for TIC-404335106-38-FFI-10min: index -1 is out of bounds for axis 0 with size 0


Error getting complex status for TIC-300651846-13-FFI-30min: index -1 is out of bounds for axis 0 with size 0
Error getting complex status for TIC-442571495-91-FFI-200s: index -1 is out of bounds for axis 0 with size 0


Error getting complex status for TIC-442575691-91-FFI-200s: index -1 is out of bounds for axis 0 with size 0


Error getting complex status for TIC-60005364-44-SPOC-120: cannot unpack non-iterable NoneType object


Error getting complex status for TIC-244161191-42-FFI-10min: index -1 is out of bounds for axis 0 with size 0


No data found for target "EPIC 60019763".


Error getting complex status for EPIC-60019763-1-K2SFF-1800: 'NoneType' object has no attribute 'to_periodogram'


In [11]:
lclist

,pop_id,lcname,complex_status
0,0,TIC-59129133-5-FFI-30min,Complex
1,0,TIC-59129133-32-FFI-10min,Complex
2,0,TIC-59129133-43-SPOC-120,Complex
3,0,TIC-59129133-44-SPOC-120,Complex
4,0,TIC-59129133-71-FFI-200s,Complex
...,...,...,...
989,194,EPIC-246932950-13-K2SFF-1800,Complex
990,195,EPIC-247857331-13-K2SFF-1800,Not Complex
991,196,EPIC-246650193-13-K2SFF-1800,Double Dip
992,200,EPIC-248131102-13-K2SFF-1800,Complex


In [17]:
lclist.sort_values(by=['pop_id','lcname'],inplace=True)

In [19]:
lclist.to_csv('lcnameswithstatus.csv',index=False)